In [1]:
from langchain.text_splitter import CharacterTextSplitter
from unstructured.partition.pdf import partition_pdf


# Extract elements from PDF
def extract_pdf_elements(path, fname):
    """
    Extract images, tables, and chunk text from a PDF file.
    path: File path, which is used to dump images (.jpg)
    fname: File name
    """
    return partition_pdf(fname,
                       extract_images_in_pdf=True,
                       infer_table_structure=True,
                       strategy="auto",
                        max_characters=4000,
                        new_after_n_chars=3800,
                        combine_text_under_n_chars=2000
    )


# Categorize elements by type
def categorize_elements(raw_pdf_elements):
    """
    Categorize extracted elements from a PDF into tables and texts.
    raw_pdf_elements: List of unstructured.documents.elements
    """
    tables = []
    texts = []
    for element in raw_pdf_elements:
        if "unstructured.documents.elements.Table" in str(type(element)):
            tables.append(str(element))
        elif "unstructured.documents.elements.CompositeElement" in str(type(element)):
            texts.append(str(element))
    return texts, tables


# File path
fpath = "/project/code/Attention_is_all_you_need.pdf"
fname = "Attention_is_all_you_need.pdf"

# Get elements
raw_pdf_elements = extract_pdf_elements(fpath, fname)

# Get text, tables
texts, tables = categorize_elements(raw_pdf_elements)

# Optional: Enforce a specific token size for texts
text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=4000, chunk_overlap=0
)
joined_texts = " ".join(texts)
texts_4k_token = text_splitter.split_text(joined_texts)

/home/workbench/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the che

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import os

# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Generate summaries of text elements
def generate_text_summaries(texts, tables, summarize_texts=False):
    """
    Summarize text elements
    texts: List of str
    tables: List of str
    summarize_texts: Bool to summarize texts
    """

    # Prompt
    prompt_text = """You are an assistant tasked with summarizing tables and text for retrieval. \
    These summaries will be embedded and used to retrieve the raw text or table elements. \
    Give a concise summary of the table or text that is well optimized for retrieval. Table or text: {element} """
    prompt = ChatPromptTemplate.from_template(prompt_text)

    # Text summary chain
    model = ChatOpenAI(temperature=0, model="gpt-4")
    summarize_chain = {"element": lambda x: x} | prompt | model | StrOutputParser()

    # Initialize empty summaries
    text_summaries = []
    table_summaries = []

    # Apply to text if texts are provided and summarization is requested
    if texts and summarize_texts:
        text_summaries = summarize_chain.batch(texts, {"max_concurrency": 5})
    elif texts:
        text_summaries = texts

    # Apply to tables if tables are provided
    if tables:
        table_summaries = summarize_chain.batch(tables, {"max_concurrency": 5})

    return text_summaries, table_summaries


# Get text, table summaries
text_summaries, table_summaries = generate_text_summaries(
    texts_4k_token, tables, summarize_texts=True
)

In [3]:
import base64
import os

from langchain_core.messages import HumanMessage


def encode_image(image_path):
    """Getting the base64 string"""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


def image_summarize(img_base64, prompt):
    """Make image summary"""
    chat = ChatOpenAI(model="gpt-4o", max_tokens=200)

    msg = chat.invoke(
        [
            HumanMessage(
                content=[
                    {"type": "text", "text": prompt},
                    {
                        "type": "image_url",
                        "image_url": {"url": f"data:image/jpeg;base64,{img_base64}"},
                    },
                ]
            )
        ]
    )
    return msg.content


def generate_img_summaries(path):
    """
    Generate summaries and base64 encoded strings for images
    path: Path to list of .jpg files extracted by Unstructured
    """

    # Store base64 encoded images
    img_base64_list = []

    # Store image summaries
    image_summaries = []

    # Prompt
    from tqdm import tqdm
    import time

    prompt = """You are an assistant tasked with summarizing images for retrieval. \
        These summaries will be embedded and used to retrieve the raw image. \
        Give a concise summary of the image that is well optimized for retrieval."""

    count = 0

    for img_file in tqdm(sorted(os.listdir(path)), desc="Processing images"):
        if img_file.endswith(".jpg"):
            img_path = os.path.join(path, img_file)

            try:
                base64_image = encode_image(img_path)
                img_base64_list.append(base64_image)
                image_summaries.append(image_summarize(base64_image, prompt))
                count += 1

            except Exception as e:
                print(f"Error processing image {img_file}: {e}")

    return img_base64_list, image_summaries


# Image summaries
img_base64_list, image_summaries = generate_img_summaries("/project/data")

Processing images: 100%|██████████| 2/2 [00:00<00:00, 32513.98it/s]


In [4]:
import uuid, os

from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain.storage import InMemoryStore
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings


def create_multi_vector_retriever(
    vectorstore, text_summaries, texts, table_summaries, tables, image_summaries, images
):
    """
    Create retriever that indexes summaries, but returns raw images or texts
    """

    # Initialize the storage layer
    store = InMemoryStore()
    id_key = "doc_id"

    # Create the multi-vector retriever
    retriever = MultiVectorRetriever(
        vectorstore=vectorstore,
        docstore=store,
        id_key=id_key,
    )

    # Helper function to add documents to the vectorstore and docstore
    def add_documents(retriever, doc_summaries, doc_contents):
        doc_ids = [str(uuid.uuid4()) for _ in doc_contents]
        summary_docs = [
            Document(page_content=s, metadata={id_key: doc_ids[i]})
            for i, s in enumerate(doc_summaries)
        ]
        retriever.vectorstore.add_documents(summary_docs)
        retriever.docstore.mset(list(zip(doc_ids, doc_contents)))

    # Add texts, tables, and images
    # Check that text_summaries is not empty before adding
    if text_summaries:
        add_documents(retriever, text_summaries, texts)
    # Check that table_summaries is not empty before adding
    if table_summaries:
        add_documents(retriever, table_summaries, tables)
    # Check that image_summaries is not empty before adding
    if image_summaries:
        add_documents(retriever, image_summaries, images)

    return retriever


# The vectorstore to use to index the summaries
vectorstore = Chroma(
    collection_name="rag-storage", embedding_function=OpenAIEmbeddings()
)

# Create retriever
retriever_multi_vector_img = create_multi_vector_retriever(
    vectorstore,
    text_summaries,
    texts,
    table_summaries,
    tables,
    image_summaries,
    img_base64_list,
)

/tmp/ipykernel_238/2114994243.py:53: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectorstore = Chroma(


In [5]:
import io
import re

from IPython.display import HTML, display
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from PIL import Image


def plt_img_base64(img_base64):
    """Disply base64 encoded string as image"""
    # Create an HTML img tag with the base64 string as the source
    image_html = f'<img src="data:image/jpeg;base64,{img_base64}" />'
    # Display the image by rendering the HTML
    display(HTML(image_html))


def looks_like_base64(sb):
    """Check if the string looks like base64"""
    return re.match("^[A-Za-z0-9+/]+[=]{0,2}$", sb) is not None


def is_image_data(b64data):
    """
    Check if the base64 data is an image by looking at the start of the data
    """
    image_signatures = {
        b"\xFF\xD8\xFF": "jpg",
        b"\x89\x50\x4E\x47\x0D\x0A\x1A\x0A": "png",
        b"\x47\x49\x46\x38": "gif",
        b"\x52\x49\x46\x46": "webp",
    }
    try:
        header = base64.b64decode(b64data)[:8]  # Decode and get the first 8 bytes
        for sig, format in image_signatures.items():
            if header.startswith(sig):
                return True
        return False
    except Exception:
        return False


def resize_base64_image(base64_string, size=(128, 128)):
    """
    Resize an image encoded as a Base64 string
    """
    # Decode the Base64 string
    img_data = base64.b64decode(base64_string)
    img = Image.open(io.BytesIO(img_data))

    # Resize the image
    resized_img = img.resize(size, Image.LANCZOS)

    # Save the resized image to a bytes buffer
    buffered = io.BytesIO()
    resized_img.save(buffered, format=img.format)

    # Encode the resized image to Base64
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def split_image_text_types(docs):
    """
    Split base64-encoded images and texts
    """
    b64_images = []
    texts = []
    for doc in docs:
        # Check if the document is of type Document and extract page_content if so
        if isinstance(doc, Document):
            doc = doc.page_content
        if looks_like_base64(doc) and is_image_data(doc):
            doc = resize_base64_image(doc, size=(1300, 600))
            b64_images.append(doc)
        else:
            texts.append(doc)
    return {"images": b64_images, "texts": texts}


def img_prompt_func(data_dict):
    """
    Join the context into a single string
    """
    formatted_texts = "\n".join(data_dict["context"]["texts"])
    messages = []

    # Adding image(s) to the messages if present
    if data_dict["context"]["images"]:
        for image in data_dict["context"]["images"]:
            image_message = {
                "type": "image_url",
                "image_url": {"url": f"data:image/jpeg;base64,{image}"},
            }
            messages.append(image_message)

    # Adding the text for analysis
    text_message = {
        "type": "text",
        "text": (
            "You are a deep learning and machine learning specialist.\n"
            "You will be given a mixed of text, tables, and image(s) usually of charts.\n"
            "Use this information to provide quality information related to the user question. \n"
            f"User-provided question: {data_dict['question']}\n\n"
            "Text and / or tables:\n"
            f"{formatted_texts}"
        ),
    }
    messages.append(text_message)
    return [HumanMessage(content=messages)]


def multi_modal_rag_chain(retriever):
    """
    Multi-modal RAG chain
    """

    # Multi-modal LLM
    model = ChatOpenAI(temperature=0, model="gpt-4o", max_tokens=1024)

    # RAG pipeline
    chain = (
        {
            "context": retriever | RunnableLambda(split_image_text_types),
            "question": RunnablePassthrough(),
        }
        | RunnableLambda(img_prompt_func)
        | model
        | StrOutputParser()
    )

    return chain


# Create RAG chain
chain_multimodal_rag = multi_modal_rag_chain(retriever_multi_vector_img)

In [6]:
# Check retrieval
query = "Can you give me a brief description on the document."
docs = retriever_multi_vector_img.get_relevant_documents(query, limit=6)

# We get 4 docs
len(docs)

/tmp/ipykernel_238/2728129363.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever_multi_vector_img.get_relevant_documents(query, limit=6)


4

In [7]:
# Run RAG chain
response = chain_multimodal_rag.invoke(query)
response = response.split('.')

# Print each line in a new line
for line in response:
    print(line)

The document appears to be a detailed comparison and analysis of various machine learning models, particularly focusing on the Transformer model and its variants
 Here is a brief description of the key points covered:

1
 **Transformer Model Configurations and Performance**:
   - The table lists different configurations of the Transformer model, including parameters like the number of layers (N), model dimension (dmodel), feed-forward dimension (dff), number of attention heads (h), and others

   - Performance metrics such as Perplexity (PPL) on the development set and BLEU scores are provided for different configurations

   - The base model configuration is highlighted, and several other configurations (A, B, C, D, E) are compared in terms of their performance


2
 **Comparison with Other Models**:
   - The document compares the BLEU scores and training costs (in FLOPs) of the Transformer model with other models like ByteNet, GNMT + RL, ConvS2S, and MoE

   - Both single models and e

In [8]:
response = chain_multimodal_rag.invoke("What is attetion mechanism?")
response = response.split(".")
for line in response:
    print (line)

The attention mechanism is a fundamental concept in modern deep learning, particularly in the context of natural language processing (NLP) and sequence modeling
 It allows models to focus on specific parts of the input sequence when making predictions, which significantly improves performance in tasks like machine translation, text summarization, and more
 Here's a detailed explanation based on the provided information:

### What is the Attention Mechanism?

The attention mechanism was introduced to address the limitations of traditional sequence models like Recurrent Neural Networks (RNNs) and Long Short-Term Memory networks (LSTMs)
 These traditional models often struggle with long-range dependencies due to their sequential nature
 Attention mechanisms, on the other hand, allow the model to weigh the importance of different parts of the input sequence dynamically


### Key Components of Attention Mechanism

1
 **Query, Key, and Value Vectors**:
   - **Query (Q)**: Represents the curr

In [9]:
response = chain_multimodal_rag.invoke("What dataset was for used for training?")
response = response.split(".")
for line in response:
    print (line)

The dataset used for training in the provided information is the Wall Street Journal (WSJ) dataset
 This is evident from multiple references to "WSJ only" in the context of discriminative training methods and semi-supervised training methods
 

Here are the specific instances where the WSJ dataset is mentioned:

1
 **Discriminative Training:**
   - Vinyals & Kaiser et al
 (2014) [37]: WSJ only, discriminative
   - Petrov et al
 (2006) [29]: WSJ only, discriminative
   - Zhu et al
 (2013) [40]: WSJ only, discriminative
   - Dyer et al
 (2016) [8]: WSJ only, discriminative
   - Transformer (4 layers): WSJ only, discriminative

2
 **Semi-Supervised Training:**
   - Zhu et al
 (2013) [40]: semi-supervised
   - Huang & Harper (2009) [14]: semi-supervised
   - McClosky et al
 (2006) [26]: semi-supervised
   - Vinyals & Kaiser et al
 (2014) [37]: semi-supervised
   - Transformer (4 layers): semi-supervised

The WSJ dataset is a well-known dataset in the field of natural language processing an

In [10]:
response = chain_multimodal_rag.invoke("What is the BLEU score for ByteNet model?")
response = response.split(".")
for line in response:
    print (line)

The BLEU score for the ByteNet model, as provided in the text, is 23
75
 This score is specifically for the English-to-German (EN-DE) translation task
 The table does not provide a BLEU score for ByteNet on the English-to-French (EN-FR) translation task
 

Here is the relevant excerpt from the table for clarity:

```
Model          BLEU EN-DE  EN-FR
ByteNet [18]   23
75
```

This indicates that ByteNet achieved a BLEU score of 23
75 for translating from English to German



In [11]:
response = chain_multimodal_rag.invoke("What is the BLEU score for ConvS2S model?")
response = response.split(".")
for line in response:
    print (line)

The BLEU score for the ConvS2S (Convolutional Sequence to Sequence) model varies depending on the language pair being translated
 Based on the provided data:

- For English to German (EN-DE), the ConvS2S model achieves a BLEU score of **25
16**

- For English to French (EN-FR), the ConvS2S model achieves a BLEU score of **40
46**


Additionally, when using an ensemble of ConvS2S models, the BLEU scores improve:

- For English to German (EN-DE), the ConvS2S Ensemble achieves a BLEU score of **26
36**

- For English to French (EN-FR), the ConvS2S Ensemble achieves a BLEU score of **41
29**


These scores indicate the performance of the ConvS2S model in terms of translation quality for the specified language pairs



In [12]:
response = chain_multimodal_rag.invoke("What part of the documnet has all the BLEU score for different models?")
response = response.split(".")
for line in response:
    print (line)

The part of the document that contains all the BLEU scores for different models is the following table:

```
Model BLEU EN-DE EN-FR Training Cost (FLOPs) EN-DE EN-FR
ByteNet [18] 23
75 
Deep-Att + PosUnk [39] 39
2 1
0 · 1020 
GNMT + RL [38] 24
6 39
92 2
3 · 1019 1
4 · 1020 
ConvS2S [9] 25
16 40
46 9
6 · 1018 1
5 · 1020 
MoE [32] 26
03 40
56 2
0 · 1019 1
2 · 1020 
Deep-Att + PosUnk Ensemble [39] 40
4 8
0 · 1020 
GNMT + RL Ensemble [38] 26
30 41
16 1
8 · 1020 1
1 · 1021 
ConvS2S Ensemble [9] 26
36 41
29 7
7 · 1019 1
2 · 1021 
Transformer (base model) 27
3 38
1 3
3 · 1018 
Transformer (big) 28
4 41
8 2
3 · 1019
```

This table lists the BLEU scores for different models for English-to-German (EN-DE) and English-to-French (EN-FR) translations, along with their respective training costs in FLOPs

